# Certificados y autoridades de certificación

En este ejercicio vamos a crear unos entidad de certificación (CA por sus siglas en inglés, *certification authority*) que sea capaz de emitir certificados.

Como en la realidad de internet, vamos a crear:

1. Una **entidad de certificación raíz**, que se encarga de emitir certificados para las entidades de certificación intermedias
1. Una **entidad de certificación intermedia**, que se encarga de emitir certificados para los usuarios finales
1. Un **certificado de usuario** final

## Certificados y claves públicas

Recuerda: en criptografía asimétrica o de clave pública como por ejemplo RSA, cada persona (identidad) tiene dos claves, una pública y una privada. La clave privada nunca debe salir del PC de la identidad, mientras que la clave pública está diseñada precisamente para eso, para hacerse pública.

¿Cómo obtenemos la clave pública de una identidad como por ejemplo una persona o un dominio web? Esa persona o web nos las envía al principio de la comunicación. ¿Y cómo sabemos que esa es la clave pública de una persona o web determinada y no de alguien que se está haciendo pasar por esa persona? Porque están firmadas digitalmente por alquien en quien todos confiamos, las *autoridades de certificación* (CA)

Un certificado es un pequeño archivo que sirve para distribuir claves públicas (por ejemplo, la clave pública RSA) de una identidad (persona o web), y puede usarse para autenticar a esa identidad.

- Un certificado es un archivo que **incluye**:
    - Detalles sobre una identidad como por ejemplo el nombre de la persona o la URL de la web para el que es válido
    - Clave pública de la identidad
    - El periodo de validez: entre una fecha y otra
    - Está firmado digitalmente por alguien que es confiable para todos: las autoridades de certificación (CA)
- Un certificado **no incluye**:
    - La clave privada de la identidad, que se tiene que guardar en un archivo aparte
- No se crea una certificado para cada conexión HTTPS o cada proceso de firma. Los certificados tienen que estar ya listos antes de que queramos usarlos. El proceso que vamos a describir se tiene que hacer una sola vez cuando se quieran empezar a usar claves públicas, y repetirlo cuando el certificado antiguo está cerca de caducar.

## Jerarquía de las autoridades de certificación

como un compromiso de una autoridad de certificación sería algo muy traumático, porque entonces un atacante podría firmar certificados y hacerse pasar por cualquiera, las autoridades de certificación se organizan en una jerarquía:

- La **autoridad de certificación raíz** (Root-CA), que no está conectada a Internet, está detás de cajas fuertes y sistemas de seguridad física y solo se enciende ocasionalmente para emitir certificados para las autoridades de certificación intermedias
- La **autoridad de certificación intermedia**, que es quien emite certificados para las personas o webs: https://support.globalsign.com/es/certificados-ca/certificados-intermedios/certificados-intermedios-informacion-general

## Usos de los certificados

Los certificados se usan siempre que necesitemos conocer la clave pública de una persona o una web.

Por ejemplo, cuando te conectas a la URL de una página web usando el protocolo TLS/HTTPS, la página web te enviará su certificado al inicio de la conexión TLS y tienes que seguir los siguientes pasos:

1. Del propio certificado, obtienes **la clave pública**, el **dominio web** para la que es válido, el **periodo de validez** y **quién lo firma (*issuer*)**.
1. Tienes que comprobar:
    1. que el dominio de la web es la que pone en el certificado
    1. que la fecha actual está dentro el periodo de validez del certificado
    1. que está firmado por una autoridad de certificación intermedia
1. Ahora haces lo mismo con la autoridad de certificación intermedia: obtienes su certificado (ya sea de la propia autoridad, o de la web), compruebas su periodo de validez y que está firmado por alguna Root-CA que esté ya instalada y reconocida por tu PC.
1. Una vez que has confirmado que el certificado de la autoridad intermedia es correcto, puedes usar la clave pública de la intermedia para validar la firma del certificado que ha presentado la web y, si todo es correcto, ¡ya tienes la clave pública de la web!

Empezamos importando las librerías que usaremos en el resto del ejercicio. `ownca` (https://ownca.readthedocs.io/en/latest/) es una librería Python que permite gestionar CAs de forma sencilla, construida sobre el paquete `cryptography` (https://cryptography.io/en/latest/), que permite acceso a más detalles. Usaremos ambos paquetes durante este ejercicio. Aún así, si quieres más detalles de todo el proceso, te animo a seguir el tutorial: https://cryptography.io/en/latest/x509/tutorial/

In [ ]:
!pip install ownca
from pprint import pprint, pformat
from textwrap import wrap

# Esta función simplemente imprime parte del certificado por pantalla. En realidad los certificados incluyen mucha más información de la que va a imprimir esta función
def print_cert(cert):
  print(f"""
  subject='{cert.cert.subject}'
  not_valid_before='{cert.cert.not_valid_before}'
  not_valid_before='{cert.cert.not_valid_before}'

  PUBLIC KEY
  ----------
  {pformat(wrap(str(cert.public_key_bytes)))}

  ISSUER
  ------
  issuer={cert.cert.issuer}

  SIGNATURE
  ---------
  {cert.cert.signature_algorithm_oid}
  {pformat(wrap(str(cert.cert.signature)))}
  """)

# Autoridad de certificación raíz: Root-CA

Creamos la autoridad de certificación raíz. La librería `ownca` guarda cada autoridad de certificación en directorio determinado. En este caso, vamos a guardar los datos de la autoridad de certificación raíz en el directorio `ROOT_CA_PATH="/opt/Root-CA"`

Además, vamos a guardar los siguientes archivos:

- `/opt/Root-CA/ca.cert`: certificado
- `/opt/Root-CA/ca_key.pem`: clave pública de la autoridad de certificación raíz. En realidad está incluida dentro del certificado, pero tenerla también en un archivo aparte hará más sencillo usarla.
- `/opt/Root-CA/private/ca_key.pem`: clave privada de la autoridad raíz. La clave privada **no está incluida** en el certificado y no debería salir nunca del PC de la Root-CA

Nota: la librería `ownca` que vamos a usar es poco flexible y necesita que los archivos se llamen exactamente así dentro del directorio `opt/Root-CA`

In [ ]:
import ownca
import os, os.path
from cryptography.hazmat.primitives import serialization

ROOT_CA_PATH = '/opt/Root-CA'

root_ca = ownca.CertificateAuthority(ca_storage=ROOT_CA_PATH, common_name='VIU-crypto Root CA')
print_cert(root_ca)

# Guardamos certificados y claves
os.makedirs(ROOT_CA_PATH, exist_ok=True)
os.makedirs(f'{ROOT_CA_PATH}/private', exist_ok=True)
with open(f'{ROOT_CA_PATH}/ca.crt', 'wb') as f:
  f.write(root_ca.cert_bytes)
with open(f'{ROOT_CA_PATH}/ca_key.pub', 'wb') as f:
   f.write(root_ca.public_key_bytes)
with open(f'{ROOT_CA_PATH}/private/ca_key.pem', 'wb') as f:
   f.write(root_ca.key_bytes)

Fíjate:

- Los certificados tienen un nombre para poder identificarlos. En este caso, "VIU-crypto Root CA"
- El certificado de una entidad de certificación raíz **también va firmado digitalmente** (campo: *issuer*), aunque las autoridades raíces pueden firmar ellas mismas sus propios certificados. Esto se llama **certificado autofirmado y solo es aceptable para autoridades de certificación raíz**.

Tú también podrías emitir tus propios certificados autofirmados por ti mismo, pero poca gente confiará en ellos. Esta es una de las causas más comunes de errores a la hora de configurar una página web: dejarse configurado el certificado autofirmado que se usa inicialmente para pruebas antes de tener el certificado real. Los navegadores webs muestan un error de seguridad cuando te intentas conectar a una web que presenta un certificado autofirmado.

# Emisión del certificado para la entidad de certificación Intermedia Interm-CA

Las autoridades de certificación raíz (Root-CAs) no están conectadas a la Intenet, esa tendría riesgo. Las Root-CAs están totalmente aisladas o incluso apagadas, y solo se encienden en momentos puntuales.

Uno de esos momentos en que se encienden es para es generar certificados para las entidades de certificación intermedias.

Bien, vamos a usar la autoridad raíz para generar (*issue*) un certificado para la CA intermedia en el directorio `INTERM_CA_PATH=/opt/Interm-CA`.

El certificado lo crea la Root-CA y vamos a guardar también estos archivos:

- `/opt/Interm-CA/ca.cert`: certificado de la autoridad intermedia
- `/opt/Interm-CA/ca_key.pem`: clave pública de la autoridad intermedia
- `/opt/Interm-CA/private/ca_key.pem`: clave privada de la autoridad intermedia

Recuerda: la librería `ownca` que vamos a usar es poco flexible y necesita que los archivos se llamen exactamente así dentro del directorio `/opt/Interm-CA`

In [ ]:
INTERM_CA_PATH = '/opt/Interm-CA'

intermediate_cert = root_ca.issue_certificate(hostname='www.viu.com', common_name='VIU-crypto Intermediate CA', ca=True)
print_cert(intermediate_cert)

# Guardamos certificados y claves
os.makedirs(INTERM_CA_PATH, exist_ok=True)
os.makedirs(f'{INTERM_CA_PATH}/private', exist_ok=True)
with open(f'{INTERM_CA_PATH}/ca.crt', 'wb') as f:
  f.write(intermediate_cert.cert_bytes)
with open(f'{INTERM_CA_PATH}/ca_key.pub', 'wb') as f:
   f.write(intermediate_cert.public_key_bytes)
with open(f'{INTERM_CA_PATH}/private/ca_key.pem', 'wb') as f:
   f.write(intermediate_cert.key_bytes)

Fíjate: en este caso, el certificado lo está generando la autoridad raíz. Por este motivo la autoridad raíz conoce también la clave privada de la autoridad intermedia y puede guardarla. Más adelante veremos cómo se generan realmente estos certificados, de forma que las autoridades no conozcan las claves privadas de las identidades.

Una vez generado el certificado, se copian los archivos generados (`/opt/Interm-CA/ca.cert`, `/opt/Interm-CA/ca_key.pem` y `/opt/Interm-CA/private/ca_key.pem`) al PC de la autoridad intermedia, por ejemplo, usando un pendrive.

El PC de la autoridad raíz ya puede apagarse completamente hasta que necesite generar otra entidad de certificación intermedia.


# Puesta en marcha de la autoridad de certificación intermedia

A partir de los archivos obtenidos en el paso anterior (el certificado y las claves pública/privada), podemos poner en marcha la autoridad de certificación intermedia.

- `/opt/Interm-CA/ca.cert`: certificado de la autoridad intermedia
- `/opt/Interm-CA/ca_key.pem`: clave pública de la autoridad intermedia
- `/opt/Interm-CA/private/ca_key.pem`: clave privada de la autoridad intermedia

**Aviso**: ls librería `ownca` es muy sencilla y necesita que los archivos tengan exactamente ese nombre en ese path, no es configurable. Otras librerías más completas te permitirán usar tu propia estructura de carpetas.

Con la siguiente línea creamos la autoridad de certificación intermedia. Esta librería lee los archivos que hemos guardado en ese directorio `INTERM_CA_PATH` durante los pasos anteriores.

In [ ]:
INTERM_CA_PATH = '/opt/Interm-CA'

intermediate_ca = ownca.CertificateAuthority(ca_storage=INTERM_CA_PATH, common_name='VIU-crypto Intermediate CA', intermediate=True)


Ahora la autoridad de certificación intermedia podría crear un certificado para un usuario, igual que lo hizo la autoridad raíz. Pero en realidad esto no funciona así: si la autoridad de certificación intermedia crea los certificados de los usuarios, tendría las claves privadas de los usuarios y hemos dicho que la clave privada **nunca debe salir del PC del usuario**

En realidad, para crear un certificado válido, se siguen los siguiente pasos:

1. El usuario crea su par de claves público/privada, por ejemplo, RSA
1. El usuario crea un archivo "solicitud de firma" que se llama CSR. Este archivo CSR incluye su identidad y la clave pública del usuario (¡pero no la privada!). El archivo CSR se envía a la autoridad de certificación intermedia.
1. La autoridad de certificación intermedia recibe el archivo CSR y valida la identidad del usuario como vea necesario
1. Cuando la autoridad de certificación intermedia ha verificado la identidad del usuario, firma los datos incluidos en el archivo CSR y crea el certificado del usuario
1. El usuario descarga o copia su certificado

Vamos paso a paso: primero, el usario crea su par de claves público/privadas y rellena el archivo CSR

# Usuario: creación de un *Certificate Signing Request* (CSR)

**Aviso**: la librería `ownca` no incluye métodos para crear archivo CSR, en esta sección usamos los métodos de la librería `cryptography`

Durante este paso, el usuario crea una solicitud de firma de certificado (CSR), incluyendo la descripción de su identidad y el par de claves público/privada. El archivo CSR va firmado por el propio usuario.

**Recuerda**: la clave privada nunca debe salir del PC del usuario, así que el archivo de clave privada no se envía a nadie y tiene que guardarse aparte. El archivo CSR no incluye la clave privada del usuario.

Para este ejemplo vamos a crear una identidad mucho más detallada que antes, incluyendo la dirección de la persona y las webs para las que el certificado debería ser válido. No lo hemos hecho para el caso de la autoridad de certificación intermedia para no complicarlo más, pero podríamos haberlo hecho también. Fíjate que esta información extra se guarda en unos campos de "extensiones". Algunos ejemplos de extensiones:

- Dominios web para los que el certificado es válido, y solo para esos
- Usos aceptados del certificado, deberá rechazarse el certificado si se intentan usar para otras cosas

Una vez creado el archivo CSR, el usuario tiene que guardar la clave privada en un fichero aparte y enviar el archivo CSR a la autoridad de certificación intermedia de su elección.

In [ ]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

# Generación de la clave privada del usuario. Recuerda que, en el caso de RSA,
# la clave privada también incluye la clave pública
user_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
# el usuario debería guardar esta clave privada y no perderla, ni enviarla a nadie

# Genera un archico CSR
user_csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Detalles de quiénes somos
    x509.NameAttribute(NameOID.COUNTRY_NAME, "ES"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Valencia"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, "Valencia"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "VIU"),
    x509.NameAttribute(NameOID.COMMON_NAME, "Nombre Usuario VIU"),
    ])
# Ejemplo de detalles adicionales: lista exclusiva de webs para las que el certificado será válido
).add_extension(
    x509.SubjectAlternativeName([
        x509.DNSName("mysite.com"),
        x509.DNSName("www.mysite.com"),
        x509.DNSName("subdomain.mysite.com"),
    ]),
    critical=False,
# Ejemplo de detalles adicionales: usos que se le dará a la clave pública
).add_extension(
    x509.KeyUsage(
        digital_signature=True,
        content_commitment=False,
        key_encipherment=True,
        data_encipherment=False,
        key_agreement=False,
        key_cert_sign=False,
        crl_sign=True,
        encipher_only=False,
        decipher_only=False,
    ),
    critical=True,
# Firma el archivo CSR con la clave privada del usuario
).sign(user_key, hashes.SHA256())

# Guarda el archivo CSR (en memoria para no complicarnos más)
user_csr_bytes = user_csr.public_bytes(serialization.Encoding.PEM)

# Autoridad intermedia: recepción del CSR y emisión del certificado

La autoridad de certificación intermedia recibe del usuario la petición de firmado (archivo CSR) y su clave pública, por los medios que haya puesto disponible al pública. En este momento, la autoridad de certificación intermedia tiene que validar la identidad del usuario, que es un proceso que puede tomar desde minutos a días. El proceso exacto depende de la propia autoridad intermedia, pero aquí tienes dos ejemplos:

- En el caso de la firma certificada con validez jurídica, es necesario que haya una identificación física del usuario. Hasta la pandemia de Covid de 2020, esto suponía que las personas tenían que desplazarse a una oficina para validar su identidad. Durante pandemia y también después se han implementado sistemas de video conferencia para probar la identidad de las personas y ya no es necesario el desplazamiento.
- Hemos visto en clase el ejemplo de Let's Encrypt, en que se usa el sistema DNS para probar que el usuario realmente controla un dominio determinado

Sea como sea, supondremos que la autoridad intermedia valida de la identidad del usuario. A continuación se emite un certificado mediente la firma digital de la iinformación que contiene el CSR.

Nota: `ownca` lanzará un error si intentas crear dos certificados para el mismo `COMMON_NAME`. En caso de que haya un error *File exists*, cambia el `COMMON_NAME` en el CSR de antes y vuelve a intentarlo.

In [ ]:
user_cert = intermediate_ca.sign_csr(user_csr, user_key.public_key(), 60)

print_cert(user_cert)

# Siguientes pasos

Ahora, la entidad intermedia remite al usuario el certificado firmado, que ya está listo para usarse. El archivo CSR ya no es necesario y tanto el usuario como la entidad certificadora intermedia pueden borrarlo.

Cómo el usario guardará el certificado dependerá de qué quiere hacer con él:

- Si es para poner en marcha una página web, tendrá que incluir en la configuración del servidor (apache o similar) la ruta del certificado y la ruta de la clave privada
- Si es para firmar digitalmente documentos, el usuario querrá incluir el certificado en su propio sistema operativo, usando las herramientas de gestión del sistema operativo
- Recuerda que los certificados tienen un periodo de validez
    - Para webs, la validez es de unos 3 meses desde el momento de la emisión
    - Para personas físicas, típicamente 2 años
    - Para autoridades intermedias, típicamente 10 años
    - Para autoridades raíz, típicamente 30 años
- Todo este proceso tiene que repetirse cuando el certificado está cerca de caducar. No podemos reutilizar las claves privadas antiguas, tenemos que crear unas nuevas. En el caso de las web, es típico que la renovación sea haga automáticamente por algún script en nuestro sistema (script que tendremos que activar y configurar correctamente)

# Preguntas

<font color="#f00">

1. Crea un certificado para ti, siguiendo los pasos de crear un CSR y que lo firme la autoridad intermedia
1. No lo hemos visto en clase, así que tendrás que buscarlo: ¿qué sucede si un usuario descubre que su clave privada se ha visto comprometida y quiere avisar al mundo que el certificado ya no es válido? Pista: *certificate revocation list*

</font>